# パラメータ設定

---

OpenHPC環境を構築するのに必要となるパラメータを設定します。

## 概要

VCP SDKを用いてクラウド上に仮想サーバを作成し、OpenHPC環境の構築を行います。

![構成](images/ohpc-000.png)

このNotebookでは以下に示すパラメータを設定します。

* VCP SDKに関するパラメータ
* VCノードに共通するパラメータ
* 割り当てリソースに関するパラメータ
    - 計算ノード
    - マスターノード
    - NFS用ディスク
* VCノードのホスト名とIPアドレス
* Slurmに関するパラメータ

## VCP SDK

VCP SDKを利用する際に必要となるパラメータを設定します。


### VCCアクセストークンの入力

![VCP SDK](images/ohpc-001.png)

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにある`unfreeze below in section`ボタンなどを利用してください。

### UnitGroup名の指定

このアプリケーションテンプレートで構築するOpenHPC環境に対して、名前を付けます。指定した名前はVCPのUnitGroup名となります。

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素  

VCP SDKで作成した他の環境と区別するために UnitGroupに名前を付けます。UnitGroup名は既存のものと異なる値を指定する必要があります。
既存のUnitGroupを確認するために一覧を表示します。

In [ ]:
vcp.df_ugroups()

この構築環境のUnitGroup名を次のセルで指定してください。UnitGroup名は**英数字のみ**の値を指定してください（先頭文字に数字は指定できない）。

> 英数字以外の文字を指定すると、環境構築時のansibleの実行にて警告文が表示される場合があります。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookでは `group_vars/` にあるファイルを `group_vars ファイル`と呼ぶことにします。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    _params=dict(vcp=vcp),
    ugroup_name=ugroup_name,
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## VCノードに共通するパラメータ

マスターノード、計算ノードに共通するパラメータを指定します。

### クラウドプロバイダの指定

VCノードを作成するプロバイダ名を指定します。

In [ ]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'
# vc_provider = 'oracle'

vc_provider = 

### SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにはsshでログインして操作を行います。sshでログインできるようにするために、事前に公開鍵をVCノードに登録します。

> 公開鍵は事前にNotebook環境で`ssh-keygen`コマンドなどを用いて作成するか、他環境で作成した鍵ファイルをNotebook環境にアップロードしておいてください。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_public_key_path = '~/.ssh/id_rsa.pub'

ssh_public_key_path = 

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_private_key_path = '~/.ssh/id_rsa'

ssh_private_key_path = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
from pathlib import Path

check_parameters(
    _params=dict(vcp=vcp),
    vc_provider=vc_provider,
    ssh_public_key_path=str(Path(ssh_public_key_path).expanduser()),
    ssh_private_key_path=str(Path(ssh_private_key_path).expanduser()),
)

この章で指定したパラメータを group_vars ファイルに保存します。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    ugroup_name,
    vc_provider=vc_provider,
    ssh_public_key_path=str(Path(ssh_public_key_path).expanduser()),
    ssh_private_key_path=str(Path(ssh_private_key_path).expanduser()),
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## 構築環境に割り当てるリソース

各VCノード、VCディスクに割り当てるリソースを指定します。

### 計算ノード

計算ノードに割り当てるリソースを指定します。

![計算ノード](images/ohpc-002.png)

#### 計算ノードのノード数

計算ノードとして作成するノード数を指定します。

In [ ]:
# (例)
# compute_nodes = 4

compute_nodes = 

#### 計算ノードのflavor

VCノードの`spec`に対して種々のパラメータを毎回設定するのは煩雑な作業になります。そこでVCP SDKでは典型的なパラメータセットを事前に定義しています。事前に定義したパラメータセットのことをVCP SDKでは`flavor`と呼んでいます。`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`もプロバイダ毎に定義されています。

次のセルを実行すると、先ほど`vc_provider`で指定したクラウドプロバイダ名に対応する `flavor` の一覧が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)

上に表示された表の `flavor` の欄の値から、計算ノードとして利用するVCノードの `flavor` を選んで次のセルで指定してください。

In [ ]:
# (例)
# compute_flavor = 'medium'
# compute_flavor = 'gpu'

compute_flavor = 

#### 計算ノードのインスタンスタイプ

`flavor`で定義されている以外のインスタンスタイプを計算ノードで利用したい場合は次のセルのコメントを外してインスタンスタイプを指定してください。

In [ ]:
# (例)
# compute_instance_type = 'g4dn.xlarge'            # AWS NVIDIA T4
# compute_instance_type = 'Standard_NC4as_T4_v3'   # Azure NVIDIA T4
# compute_instance_type = 'VM.GPU2.1'              # Oracle Cloud  NVIDIA P100

#### 計算ノードのルートボリュームサイズ

計算ノードのルートボリュームサイズを変更する必要がある場合は、次のセルのコメントを外してサイズ(GB)を指定してください。

計算ノードに Docker Engine をインストールして利用することを予定している場合は十分大きなサイズを指定してください。各計算ノードの`/var/lib/docker`はルートボリュームに割り当てたディスクの領域を利用します。

In [ ]:
# (例)
# compute_root_size = 16

#### 計算ノードにおけるGPUの利用

計算ノードでGPUを利用するか否かについて指定してください。

In [ ]:
# (例)
# compute_use_gpu = False  # GPU を利用しない
# compute_use_gpu = True   # GPU を利用する

compute_use_gpu = 

`compute_use_gpu`を`True`に設定することで、計算ノードを起動する際のVMイメージ、BaseコンテナイメージとしてNVIDIAドライバがセットアップされているものが選択されます。ただし`compute_flavor`や`compute_instance_type`で指定したインスタンスタイプがGPUを利用可能なものである必要があります。

### マスターノード

マスターノードに割り当てるリソースを指定します。

![マスターノード](images/ohpc-003.png)

#### マスターノードのflavor

マスターノードに割り当てるリソースに対応する `flavor` の値を指定してください。

`flavor`の一覧を表示します。

In [ ]:
vcp.df_flavors(vc_provider)

上に表示された表の `flavor` の欄の値から、マスターノードとして利用するVCノードの `flavor` を選んで次のセルで指定してください。

In [ ]:
# (例)
# master_flavor = 'small'

master_flavor = 

#### マスターノードのインスタンスタイプ

`flavor`で定義されている以外のインスタンスタイプをマスターノードで利用したい場合は次のセルのコメントを外してインスタンスタイプを指定してください。

In [ ]:
# (例)
# master_instance_type = 'm5.xlarge'

#### マスターノードのルートボリュームサイズ

マスターノード用VCノードのルートボリュームサイズ(GB)を指定します。

OpenHPCのマスターノードのルートボリュームサイズには20GB以上の値を指定してください。また NGC カタログのコンテナを利用する場合は Singularity でコンテナイメージを変換するための作業領域が必要となるため 60GB 以上の値を指定してください。

In [ ]:
# (例)
# master_root_size = 60

master_root_size = 

### NFS用ディスク

NFS用ディスクに割り当てるリソースを指定します。

> NFS用に仮想ディスクを作成しない構成にする場合は、この節をスキップしてください。

![NFS](images/ohpc-004.png)

ディスクサイズ(GB)を指定してください。16GB 以上の値を指定してください。

In [ ]:
# (例)
# nfs_disk_size = 64

nfs_disk_size = 

ディスクのデバイス名を指定してください。デバイス名はプロバイダやインスタンスタイプによって異なる値となります。

> AWSの[Nitroベースのインスタンス](https://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/instance-types.html#ec2-nitro-instances)ではEBSボリュームはNVMeブロックデバイスとして公開されます。そのためデバイス名は`/dev/nvme1n1`などになります。

In [ ]:
# (例)
# nfs_device = '/dev/xvdf'    # AWS
# nfs_device = '/dev/nvme1n1' # AWS(Nitroベース)
# nfs_device = '/dev/sdc'     # Azure
# nfs_device = '/dev/sdb'     # Oracle Cloud

nfs_device = 

NFS用に仮想ディスクを作成しない構成の場合、誤ってディスクに関するパラメータを設定してしまった場合は、次のセルのコメントを外して実行してください。

In [ ]:
# del(nfs_disk_size)
# del(nfs_device)

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
from pathlib import Path

if 'nfs_disk_size' in vars():
    check_parameters(
        _params=dict(vc_provider=vc_provider),
        nfs_disk_size=nfs_disk_size,
        nfs_device=nfs_device,
    )

check_parameters(
    _params=dict(
        vcp=vcp, 
        vc_provider=vc_provider,
        nfs_disk_size=nfs_disk_size if 'nfs_disk_size' in vars() else 0,
    ),
    compute_nodes=compute_nodes,
    compute_flavor=compute_flavor,
    master_flavor=master_flavor,
    master_root_size=master_root_size,
)

この章で指定したパラメータを `group_vars` ファイルに保存します。

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    compute_nodes=compute_nodes,
    compute_flavor=compute_flavor,
    compute_use_gpu=compute_use_gpu,
    master_flavor=master_flavor,
    master_root_size=master_root_size,
)

if 'compute_instance_type' in vars():
    update_group_vars(
        ugroup_name, 
        compute_instance_type=compute_instance_type,
    )

if 'compute_root_size' in vars():
    update_group_vars(
        ugroup_name, 
        compute_root_size=compute_root_size,
    )
    
if 'master_instance_type' in vars():
    update_group_vars(
        ugroup_name, 
        master_instance_type=master_instance_type,
    )

if 'nfs_disk_size' in vars():
    update_group_vars(
        ugroup_name,
        nfs_disk_size=nfs_disk_size,
        nfs_device=nfs_device,
    )

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## IPアドレスとホスト名

各ノードに設定するIPアドレスとホスト名を指定します。

![etc_hosts](images/ohpc-005.png)

### マスターノードのIPアドレスとホスト名

マスターノードに割り当てるIPアドレスを指定します。

まず、VCノードに割り当て可能なIPアドレスの範囲を確認します。

In [ ]:
print(vcp.get_vpn_catalog(vc_provider).get('private_network_ipmask'))

次のセルに、マスターノードに割り当てるIPアドレスを指定してください。上のセルの出力結果に表示された範囲のIPアドレスを指定してください。

In [ ]:
# (例)
# master_ipaddress = '172.30.2.120'

master_ipaddress = 

マスターノードに設定するホスト名を指定してください。

In [ ]:
# (例)
# master_hostname = 'master'

master_hostname = 'master'

### 計算ノードのIPアドレスとホスト名

計算ノードに割り当てるIPアドレスとホスト名を指定します。

計算ノードの先頭のIPアドレスを指定してください。他のノードは連番のアドレスが指定されます。指定する値は、マスターノードのIPアドレスを指定するときに確認した範囲のIPアドレスを指定してください。

> 計算ノードに連番ではなく任意のIPアドレスを割り当てる場合は、この節の最後にある記述を確認してください。

In [ ]:
# (例)
# c_ip_address = '172.30.2.121'

c_ip_address = 

計算ノードのホスト名のプレフィックスを指定してください。

例えば `c_hostname_prefix` に `c` を指定した場合、計算ノードのホスト名は以下のようになります。

* c1
* c2
* c3
* ...

In [ ]:
# (例)
# c_hostname_prefix = 'c'

c_hostname_prefix = 'c'

ここまでに指定したパラメータから計算ノードのIPアドレスとホスト名のdict型変数を作成します。

In [ ]:
import ipaddress

c_addr = ipaddress.ip_address(c_ip_address)
compute_etc_hosts = dict([
    (f'{c_addr + x}', f'{c_hostname_prefix}{x + 1}')
    for x in range(compute_nodes)   
])

import json
print(json.dumps(compute_etc_hosts, indent=2))

計算ノードのIPアドレスに連番ではなく、任意のアドレスを指定したい場合は`compute_etc_hosts`の値としてIPアドレスをキー、ホスト名を値とする dict を指定してください。

```
compute_etc_hosts = {
    "172.30.2.111": "c1",
    "172.30.2.121": "c2",
    "172.30.2.131": "c3",
    "172.30.2.141": "c4",
}
```

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

> IPアドレスに到達可能なものがあるかをチェックするので、ある程度時間が掛かります。

In [ ]:
%run scripts/utils.py

check_parameters(
    _params=dict(
        vcp=vcp,
        vc_provider=vc_provider,
        compute_nodes=compute_nodes,
    ),
    master_ipaddress=master_ipaddress,
    master_hostname=master_hostname,
    compute_etc_hosts=compute_etc_hosts,
)

この章で指定したパラメータを `group_vars` ファイルに保存します。

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    master_ipaddress=master_ipaddress,
    master_hostname=master_hostname,
    compute_etc_hosts=compute_etc_hosts,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## Slurm

Slurmに関連するパラメータを指定します。

![etc_hosts](images/ohpc-006.png)

### munge.key

[MUNGE](https://dun.github.io/munge/)はHPCクラスタ環境のための認証サービスです。この節ではSLURMがコンポーネント間の認証に利用するMUNGEの鍵ファイル`munge.key`を作成します。

`munge.key` に書き込む内容を乱数から生成します。

In [ ]:
import secrets

munge_key = secrets.token_bytes(1024)

`munge.key` の内容は秘匿情報になるので、`group_vars`ファイルではなく VC Controller の HashiCorp Vault に保存します。
HashiCorp Vault は秘密情報を保存するための Key Valueストアです。保持する情報は暗号化されます。


HashiCorp Valutのなかの記録場所となるパスを次のセルで指定します。

In [ ]:
vault_path_munge_key = f'cubbyhole/OpenHPC/{ugroup_name}/munge.key'
print(vault_path_munge_key)

### slurm.conf

`slurm.conf` に登録する計算ノードのホスト名を指定します。

計算ノードとして利用するホスト名を `slurm.conf` に登録する必要があります。
次のセルを実行すると「[5.2  計算ノードのIPアドレスとホスト名](#計算ノードのIPアドレスとホスト名)」で指定した値に従い、`NodeName` の値を設定します。

> ただしIPアドレスに任意の値を指定した場合は、次のセルを実行するとエラーとなることがあります。その場合、指定したホスト名に合わせて設定を行ってください。

In [ ]:
slurm_conf = {
    'NodeName': f'{c_hostname_prefix}[1-{compute_nodes}]' if compute_nodes > 1 else f'{c_hostname_prefix}1',
# (例)
#    'CPUs': 4,
#    'Boards': 1,
#    'SocketsPerBoard': 1,
#    'CoresPerSocket': 2,
#    'ThreadsPerCore': 2,
#    'RealMemory': 16045,
}

`slurm_conf`に設定した値を確認します。

In [ ]:
import json
print(json.dumps(slurm_conf, indent=2))

`slurm_conf`では`NodeName` に対応するノードの`CPUs`などのリソース量をあわせて指定することもできます。

> リソース量についてはOpenHPC環境を構築した後に編集し直すことができます。ここではリソース量を指定せずに `NodeName` の指定だけでも問題ありません。

例えば `slurm_conf` に
```
{
  "NodeName": "c[1-4]",
  "CPUs": 4
  "Boards": 1,
  "SocketsPerBoard": 1,
  "CoresPerSocket": 2,
  "ThreadsPerCore": 2,
  "RealMemory": 16045
}
```
を指定した場合、設定ファイル`slurm.conf`に以下のような行が追加されます。

```
NodeName=c[1-4] CPUs=4 Boards=1 SocketsPerBoard=1 CoresPerSocket=2 ThreadsPerCore=2 RealMemory=16045
```

`slurm.conf`の設定方法の詳細については[slurm.conf(5) - NODE CONFIGURATION](https://slurm.schedmd.com/slurm.conf.html#SECTION_NODE-CONFIGURATION)を参照してください。

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。`munge.key`は秘匿情報のため、暗号化され記録される HashiCorp Vaultに保存します。他のパラメータについては `group_vars`ファイルに保存します。

`munge.key`の内容をVCCのHashiCorp Valutに保存します。次のセルを実行してください。

> 保存に成功すると `<Response [204]>` と表示されます。

In [ ]:
import requests
import base64

payload = {
    'munge.key': base64.b64encode(munge_key).decode('UTF-8'),
}

vault_url = f'{vcp.vcc_info()["vault_url"]}/v1/{vault_path_munge_key}'

custom_headers = {
    'X-Vault-Token': vcc_access_token,
}

r = requests.post(vault_url, headers=custom_headers, json=payload)
r

他の値を `group_vars`ファイルに保存します。

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    vault_path_munge_key=vault_path_munge_key,
    slurm_conf=slurm_conf,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## チェック

設定項目漏れがないことを確認します。

次のセルを実行しエラーとならないことを確認してください。エラーになった場合は、このNotebookの中に実行していないセルがないかを確認してください。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
require_params = [
    'compute_etc_hosts', 'compute_flavor', 'compute_nodes',
    'compute_use_gpu', 'master_flavor', 'master_hostname',
    'master_ipaddress', 'master_root_size', 'slurm_conf',
    'ugroup_name', 'ssh_private_key_path', 'ssh_public_key_path',
    'vault_path_munge_key', 'vc_provider',
]

for x in require_params:
    if x not in gvars:
        raise RuntimeError("ERROR: not set {}".format(x))